In [41]:
import pandas as pd
from math import sqrt

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from pathlib import Path

#read CSV file
directory = '/eos/user/m/mkhalife/eos/newRes/CSV'
csvFileName = 'combinedZeroBias22G.csv'
eraName = 'AllG'
df = pd.read_csv('{0}/{1}'.format(directory , csvFileName))
df

,Unnamed: 0,"Lumi bin 1= (0.006 , 9.989)","Lumi bin 2= (9.989 , 15.645)","Lumi bin 3= (15.645 , 17.403)","Lumi bin 4= (17.403 , 21.852)","Lumi bin 5= (21.852 , 25.701)","Lumi bin 6= (25.701 , 40.425)","Lumi bin 1= (0.006 , 9.989) .1","Lumi bin 2= (9.989 , 15.645) .1","Lumi bin 3= (15.645 , 17.403) .1","Lumi bin 4= (17.403 , 21.852) .1","Lumi bin 5= (21.852 , 25.701) .1","Lumi bin 6= (25.701 , 40.425) .1"
0,nVertices,7.281236,4.932103,3.214232,2.149977,0.894007,0.784973,99.0,99.0,82.0,81.0,76.0,77.0
1,nGoodVertices,6.551879,4.886898,3.284721,2.028575,0.864354,0.709093,99.0,90.0,74.0,73.0,69.0,70.0
2,nEles,3.796307,0.551806,0.020770,0.096276,0.227677,0.466275,99.0,99.0,84.0,79.0,77.0,73.0
3,nMus,11.301400,5.321659,5.011651,2.551939,6.955850,9.365867,99.0,99.0,80.0,99.0,99.0,99.0
4,nChargedHadrons,3.616436,1.414988,1.110289,0.907204,0.793324,0.812430,99.0,94.0,78.0,74.0,70.0,70.0
5,nLostTracks,6.554829,4.800710,2.669603,1.121655,0.732388,0.932764,99.0,99.0,99.0,99.0,94.0,92.0
6,nPhotons,5.586154,1.518242,1.374974,1.162529,1.047478,1.181748,99.0,93.0,71.0,72.0,70.0,68.0
7,nNeutralHadrons,3.170307,3.562945,6.776515,2.413255,1.863101,0.999809,99.0,99.0,99.0,99.0,99.0,98.0
8,fixedGridRhoAll,6.411424,2.069694,2.148370,1.471593,2.130539,1.809754,99.0,99.0,85.0,82.0,77.0,76.0
9,fixedGridRhoFastjetAll,7.070984,2.159300,2.416498,1.650898,2.612677,2.095920,99.0,99.0,84.0,83.0,78.0,77.0


In [42]:
#make an entire integer dataframe
columnToRemove = 'Unnamed: 0'
dfRemoved = df.drop(columns = columnToRemove)
#dfRemoved

In [43]:
#remove nEles = 2, nMus = 3, nLostTracks = 5, nNeutralHadrons = 7 and fixedGridRhoAll = 8.
#for i in (2 , 3 , 5 , 7 , 8):
#    indexToRemove = i
#    df = df.drop(indexToRemove)
#df

In [44]:
#select propriate rows which have
#at least 3 lumi bins with fit quality less than 11
#and at least 3 lumi bins with cross section less than 90

#condition1 = (dfRemoved.iloc[: , 0:6] < 11).sum(axis=1) >= 3
#condition2 = (dfRemoved.iloc[: , 7:13] < 90).sum(axis=1) >= 3

#selRows = df[condition1 & condition2]
#selRows

In [45]:
#calculate cross section divided by fit quality

fit = []
cross = []
for columnIndex in range (1,7):
    fitValue = df.iloc[:, columnIndex]
    fit.append(fitValue)
    
    crossValue = df.iloc[:, columnIndex + 6]
    cross.append(crossValue)

    
seriesName = [s.name for s in fit]
crossFit = [a / b for a,b in zip(cross , fit)]


for i, fitSeries in enumerate(fit):
    crossFit[i].name = fitSeries.name.replace("fit", "crossFit")

for i, fitSeries in enumerate(fit):
    cross[i].name = fitSeries.name.replace("fit", "cross")

In [46]:
#cross[4].values[6]
#cross[0].name

In [47]:
#type(cross)

In [48]:
#fit

In [49]:
#crossFit

In [50]:
#convert lists to dataframe
dfF = pd.DataFrame(fit).T
dfFInv = 1/dfF
dfC = pd.DataFrame(cross).T
dfCperF = pd.DataFrame(crossFit).T
#dfF
#dfFInv
#dfC
#dfCperF

In [51]:
#create dataframe
dfFinal = pd.concat([dfFInv, dfCperF], axis=0)

In [52]:
#dfFinal

In [53]:
#calculate mean
sumFitInv = dfFinal.iloc[0:len(df)].sum()
dfFinal.loc['Sum Fit Inverse'] = sumFitInv
sumCperF = dfFinal.iloc[len(df):len(df)*2].sum()

dfFinal.loc['Sum Cross section per Fit quality'] = sumCperF
dfFinal.loc['Cross section Weighted Mean'] = dfFinal.loc['Sum Cross section per Fit quality'] / dfFinal.loc['Sum Fit Inverse']
len(df)

15

In [54]:
#dfFinal

In [55]:
#cross section, fit inverse and weighted mean dataframe

dfVar = pd.concat([dfC , dfFInv], axis=0)
dfVar = dfVar.append(dfFinal.loc['Cross section Weighted Mean'] , ignore_index=True)
#dfVar

/tmp/ipykernel_985/4042084114.py:4: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [56]:
#calculate variance

weightedSum = []
for i in range(len(df)):
    x = (dfVar.iloc[i + len(df)]) * ((dfVar.iloc[i] - dfVar.iloc[-1]) * (dfVar.iloc[i] - dfVar.iloc[-1]))
    weightedSum.append(x)
    
s = pd.DataFrame(weightedSum)

sumS = s.iloc[:].sum()
sumS.name = 'Sum of wightes * (x_i - xbar)^2'

sumF = dfFinal.loc['Sum Fit Inverse']

s = s.append(sumS) # , ignore_index = True)
s = s.append(sumF)
#type(s.loc['Sum Fit Inverse'].iloc[0])
index = s.columns
#s

/tmp/ipykernel_985/2811156606.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_985/2811156606.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [57]:
factor = (len(df)) /(len(df) - 1)

variance = []
for i in range (0 , 6):
    ssLocal = sqrt(factor * ((s.loc['Sum of wightes * (x_i - xbar)^2'].iloc[i]) / (s.loc['Sum Fit Inverse'].iloc[i])))
    variance.append(ssLocal)
    
#variance

Variance = pd.DataFrame(variance).T
Variance.shape
Variance.rename(index={0: 'Variance'}, inplace=True)
#VV = pd.concat()
#pd.set_columns(Variance) = keys
Variance.columns = index
#Variance

In [58]:
CS_WM = pd.DataFrame (dfFinal.loc['Cross section Weighted Mean']).T
whole = pd.concat([s , CS_WM , Variance])

In [59]:
whole

,"Lumi bin 1= (0.006 , 9.989)","Lumi bin 2= (9.989 , 15.645)","Lumi bin 3= (15.645 , 17.403)","Lumi bin 4= (17.403 , 21.852)","Lumi bin 5= (21.852 , 25.701)","Lumi bin 6= (25.701 , 40.425)"
0,1.109418e-28,0.429916,1.079071,0.278292,8.477908,1.208710
1,1.232919e-28,11.645339,29.611694,37.945117,110.049941,89.671944
2,2.127840e-28,3.842632,0.912136,79.899060,13.498052,53.061644
3,7.147730e-29,0.398445,2.976628,116.284884,58.934390,47.202236
4,2.233673e-28,8.875578,30.953440,66.608498,96.575918,78.266129
5,1.232364e-28,0.441682,85.836044,264.566142,317.412644,210.907367
6,1.446064e-28,13.598964,120.322501,82.167040,73.143290,84.182110
7,2.547998e-28,0.595123,33.815046,122.967477,220.030321,401.114433
8,1.259928e-28,1.024494,0.602422,0.034858,1.442452,2.153298
9,1.142406e-28,0.981980,0.007840,0.911184,0.217054,0.452692


In [60]:
#len(X) = len(index)
cmscs = 69.2
Y = [cmscs]* len(index)
y_upper = [cmscs + cmscs*4.6/100]*len(index)
y_lower = [cmscs - cmscs*4.6/100]*len(index)

fig = go.Figure([
    go.Scatter(
        x=index,
        y=whole.iloc[-2],
        name = 'Cross Section' ,
        line=dict(color='red'),
        error_y=dict(
            type='data',
            array=whole.iloc[-1],
            visible=True)
    )]
    )

custom_template = {
    "layout": go.Layout(
        font={
            "family": "Nunito",
            "size": 16,
            "color": "black",
        },
        title={
            "text": "Cross section",
            "font": {
                "family": "Lato",
                "size": 18,
                "color": "#1f1f1f",
            },
        },
        autosize=False,
        width=1000,
        height=500,
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff",
        colorway=px.colors.qualitative.G10,
        legend=dict(font=dict(family="Nunito", size=12, color="black")),
    )
}
fig.add_scatter(x = index, y = Y , name = 'CMS cross section' , line=dict(color='rgb(0,100,80)'))
fig.add_scatter(x = index , y = y_upper , name = 'Error Band' , line=dict(color='rgb(0,100,80)') , fill='tonexty')
fig.add_scatter(x = index , y = y_lower , name = 'Error Band' , line=dict(color='rgb(0,100,80)') , fill='tonexty')

fig.update_layout(custom_template['layout'])
fig.show()

ghtml = fig.to_html()
Path('{0}/'.format(directory) ).mkdir(parents=True, exist_ok=True)
with open('{0}/{1}.html'.format(directory , eraName) , 'w') as f:
    f.write(ghtml)